In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torchvision
import numpy as np
from torch.utils.data import DataLoader
import mne
import matplotlib.pyplot as plt

In [160]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(10, 1, 55) # CNN1
        self.pool = nn.MaxPool2d(1, 16)
        self.conv2 = nn.Conv2d(5, 1, 25) # CNN2
        self.batch = nn.BatchNorm2d()
        self.dense1 = nn.Linear(10) # dense
        self.dense2 = nn.Linear(5) # dense
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(self.batch(F.relu(self.conv1(x)))) # CNN1
        x = self.pool(self.batch(F.relu(self.conv2(x)))) # CNN2
        x = self.flatten(x)
        x = F.relu(self.dense1(x)) # relu activation
        x = F.softmax(self.dense2(x)) # softmax activation
        return x

In [1]:
import torch
import numpy as np
A = np.array([1, 2, 3])
B = np.array([1, 2, 4])
A, B = torch.Tensor(A), torch.Tensor(B)
print(torch.sum(A==B))

tensor(2)


In [161]:
## Debug function set_generate
# first generate pseudo-training data
def set_generate(x):  # input NxM signal
    set_all = []
    N, M = x.shape # M is the number of data points of each epoch, N is the number of epoch
    x_0 = x[0, :] # data of the first epoch 
    x_f = x[-1, :] # data of the last epoch
    x_current = x # current set
    set_all.append(x_current)
    x_previous = np.roll(x, 1, axis=0)
    x_previous[0, :] = x_0  # ''previous'' set
    set_all.append(x_previous)
    x_next = np.roll(x, -1, axis=0)
    x_next[0, :] = x_f # ''next'' set
    set_all.append(x_next)
    set_all = torch.Tensor(np.array(set_all))
    x_present, x_previous, x_next = set_all
    return x_present, x_previous, x_next
x = np.random.rand(6, 5)
x_present, x_previous, x_next = set_generate(x)

#x = torch.Tensor(x)
#y = np.array([1,2,3,4,2,1]).reshape(-1,1)
#y = torch.Tensor(y)
print(x_present.shape)
print(x_present)
print(x_previous)

torch.Size([6, 5])
tensor([[0.6751, 0.6729, 0.8042, 0.1923, 0.0071],
        [0.5162, 0.9818, 0.1838, 0.1339, 0.0855],
        [0.5879, 0.9085, 0.5527, 0.0845, 0.6501],
        [0.3729, 0.6553, 0.9589, 0.4340, 0.5202],
        [0.4263, 0.7769, 0.1532, 0.5452, 0.7224],
        [0.2034, 0.7723, 0.0827, 0.4716, 0.7123]])
tensor([[0.6751, 0.6729, 0.8042, 0.1923, 0.0071],
        [0.6751, 0.6729, 0.8042, 0.1923, 0.0071],
        [0.5162, 0.9818, 0.1838, 0.1339, 0.0855],
        [0.5879, 0.9085, 0.5527, 0.0845, 0.6501],
        [0.3729, 0.6553, 0.9589, 0.4340, 0.5202],
        [0.4263, 0.7769, 0.1532, 0.5452, 0.7224]])


In [4]:
## Debug CNN model
# prepare a 4 x 3000 dataset for debugging
x_debug = torch.rand(6, 1, 1, 3000) 
# N, M = x_debug.shape
events_train = np.array([1,1,2,4,3,3]).reshape(-1,1)
y_debug = np.array([1,1,2,4,3,3]).reshape(-1,1)
y_debug = torch.Tensor(events_train)
train_data = list(zip(x_debug, events_train)) 
train_loader = DataLoader(train_data)
print(train_data)
m = nn.Conv2d(1, 10 , (1, 55), padding='same')
output = m(x_debug)
m = nn.BatchNorm2d(10)
output = m(output)
m = nn.MaxPool2d(kernel_size=(1, 16), ceil_mode=True)
output = m(output)
m = nn.Conv2d(10, 5 , (1, 25), padding='same')
output = m(output)
m = nn.MaxPool2d(kernel_size=(1, 16), ceil_mode=True)
output = m(output)
m = nn.Flatten()
output = m(output)
m = nn.Linear(output.shape[1], 10)
output = m(output)
output = F.relu(output)
m = nn.Linear(10, 5)
output = m(output)
m = nn.Softmax(dim=0)
output = m(output)
print(output.shape)

# from here, we get 5 features

[(tensor([[[0.7914, 0.2609, 0.6006,  ..., 0.0307, 0.1315, 0.1159]]]), array([1])), (tensor([[[0.1312, 0.2037, 0.1271,  ..., 0.7571, 0.1158, 0.8011]]]), array([1])), (tensor([[[0.8774, 0.4772, 0.8136,  ..., 0.6337, 0.7330, 0.2619]]]), array([2])), (tensor([[[0.5954, 0.1356, 0.7059,  ..., 0.3260, 0.1988, 0.1966]]]), array([4])), (tensor([[[0.7287, 0.3694, 0.1642,  ..., 0.7542, 0.5326, 0.8044]]]), array([3])), (tensor([[[0.2945, 0.6379, 0.5975,  ..., 0.2747, 0.4902, 0.0801]]]), array([3]))]
torch.Size([6, 5])


In [163]:
y_debug = np.array([0,1,1,2,3,3,4,4])
y_debug = torch.Tensor(y_debug)
x_debug = torch.rand(8, 5)


print(x_debug)
#print(indx)
#print(x_debug[indx_row, ind_col])
loss = 0

events_train = np.array([0,1,1,2,3,3,4,4])
N = len(events_train)
Nc = np.sum(events_train == 1)
p1 = Nc/N
p2 = 1 - p1
W = np.zeros(N)
W[:] = p2
ind_corres = np.where(events_train == 1)
W[ind_corres] = p1
w = 0.5 / W
w = torch.tensor(w)
print(w.shape)


indx_row = np.arange(N).reshape(-1, 1)
indx_col = events_train.reshape(-1, 1)

#for i in range(5): # class i
#    indx_row = torch.nonzero(target.int() == i)
#    indx_col = i
#    w = self.weight
#    w = w[indx_row]
#    subloss = -torch.sum(torch.log(w * predictions[indx_row, indx_col]))
#    loss = loss + subloss
#    loss = loss / len(target)

#for i in range(5):
#    indx_row = torch.nonzero(y_debug.int() == i)
#    indx_col = i
#    weight = w[indx_row]
#    subloss = -torch.sum(torch.log(weight * x_debug[indx_row, indx_col]))
#    loss = loss + subloss

#print(weight)
#print(x_debug[indx_row, indx_col])
#print(weight * x_debug[indx_row, indx_col])
#print(loss)
#loss = loss/len(y_debug)
#print(loss)


tensor([[0.3982, 0.4006, 0.2288, 0.9587, 0.9813],
        [0.8011, 0.1549, 0.3968, 0.9406, 0.8389],
        [0.7264, 0.6093, 0.8447, 0.8673, 0.1460],
        [0.1324, 0.2173, 0.2836, 0.6652, 0.4339],
        [0.8869, 0.2896, 0.3612, 0.3382, 0.2279],
        [0.1695, 0.9622, 0.0634, 0.8426, 0.6103],
        [0.7545, 0.7025, 0.1083, 0.5020, 0.2746],
        [0.3001, 0.7268, 0.6771, 0.7439, 0.9983]])
torch.Size([8])


In [164]:
# Debug BiLSTM model
## model:
class BiLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(75, 128, bidirectional=True)
        self.dense = nn.Linear(256, 5)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dense(x)
        x = F.softmax(x)
        return x
    

x_debug = torch.rand(6, 75)
m = nn.LSTM(75, 128, bidirectional=True)
output, _ = m(x_debug)
print(output.shape)
m = nn.Linear(256, 5)
output = m(output)
output = F.softmax(output, dim=1)
print(output.shape)

torch.Size([6, 256])
torch.Size([6, 5])


In [165]:
## Debug Weight for each submodels, take N1-N3 as example
submodel_name = np.array(['N1', 'N2', 'N3'])
labels_example = np.array(['N1', 'N2', 'N1', 'N2', 'N3']) # example labels with size = 5.
N = len(labels_example)
i = 0

for stage in submodel_name:
    Nc = np.sum(labels_example==stage)
    p1 = Nc / N # probability of 'stage' in the corresponding model
    p2 = 1 - p1 # probability of other 'stage' in this model
    #p[:] = p2
    #p[i] = p1
    #w = 0.5 / p # weight for each stage in this sub-model
    W = np.zeros(N) # initialize weight vector
    W[:] = p2 
    ind_corres = np.where(labels_example==stage) # get the index of stages of corresponding sub-model
    W[ind_corres] = p1
    W = 0.5 / W
    W = torch.Tensor(W)
    i = i+1
    #print(stage, 'sub-model:', submodel_name, ':', w)
    print(stage, 'sub-model', labels_example, ':', W)

N1 sub-model ['N1' 'N2' 'N1' 'N2' 'N3'] : tensor([1.2500, 0.8333, 1.2500, 0.8333, 0.8333])
N2 sub-model ['N1' 'N2' 'N1' 'N2' 'N3'] : tensor([0.8333, 1.2500, 0.8333, 1.2500, 0.8333])
N3 sub-model ['N1' 'N2' 'N1' 'N2' 'N3'] : tensor([0.6250, 0.6250, 0.6250, 0.6250, 2.5000])


In [166]:
# Debug loading system
import mne
alice_files = ["G:\My Drive\Sleep\sleep-edf-database-expanded-1.0.0\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4001E0-PSG.edf",
               "G:\My Drive\Sleep\sleep-edf-database-expanded-1.0.0\sleep-edf-database-expanded-1.0.0\sleep-cassette\SC4001EC-Hypnogram.edf"]
raw_train = mne.io.read_raw_edf(
    alice_files[0],
    stim_channel="Event marker",
    infer_types=True,
    preload=True,
    verbose="error",  # ignore issues with stored filter settings
)
annot_train = mne.read_annotations(alice_files[1])

raw_train.set_annotations(annot_train, emit_warning=False)

# plot some data
# scalings were chosen manually to allow for simultaneous visualization of
# different channel types in this specific dataset

annotation_desc_2_event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 4,
    "Sleep stage R": 5,
}

# keep last 30-min wake events before sleep and first 30-min wake events after
# sleep and redefine annotations on raw data
annot_train.crop(annot_train[1]["onset"] - 30 * 60, annot_train[-2]["onset"] + 30 * 60)
raw_train.set_annotations(annot_train, emit_warning=False)

events_train, _ = mne.events_from_annotations(
    raw_train, event_id=annotation_desc_2_event_id, chunk_duration=30.0
)

# create a new event_id that unifies stages 3 and 4
event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3/4": 4,
    "Sleep stage R": 5,
}

tmax = 30.0 - 1.0 / raw_train.info["sfreq"]  # tmax in included

epochs = mne.Epochs(
    raw=raw_train,
    events=events_train,
    event_id=event_id,
    tmin=0.0,
    tmax=tmax,
    baseline=None,
)
# del raw_train
epochs.info
print(epochs.get_data().shape)
epochs_train = epochs.get_data()[:,0,:]
events_train = events_train[:, -1].reshape(-1,1)


Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 841 events and 3000 original time points ...
0 bad epochs dropped
(841, 7, 3000)
Using data from preloaded Raw for 841 events and 3000 original time points ...
(841, 3000)
(841, 1)
<RawEDF | SC4001E0-PSG.edf, 7 x 7950000 (79500.0 s), ~424.6 MB, data loaded>
(842, 3000)
(842, 1)
(841, 3000)
(841, 1)


In [167]:
import pandas as pd
import numpy as np
import mne
from mne.io import read_raw_edf, concatenate_raws 

path_psg = pd.read_excel(io='filepath-psg.xlsx', header=None)
path_psg = list(np.ravel(path_psg.values.tolist()))

path_hyp = pd.read_excel(io='filepath-hyp.xlsx', header=None)
path_hyp = list(np.ravel(path_hyp.values.tolist()))


#raws = [read_raw_edf(file, preload=True) for file in path_psg]
#data = concatenate_raws(raws)


FileNotFoundError: [Errno 2] No such file or directory: 'filepath-psg.xlsx'

In [6]:
# Loss Function Revised Version 
import numpy as np
import torch
x_debug = torch.rand(8,5)
N, c = x_debug.shape
print(x_debug)
label = np.array([0, 0, 1, 1, 2, 2, 3, 4])
label = torch.Tensor(label)
print(label)
indx_row = torch.Tensor(np.arange(N)).int()
indx_col = label.int()
indx_features = (indx_row, indx_col)
print(indx_features)
predictions = x_debug[indx_features]
w = torch.zeros(N)
print(w)
print(w * predictions)

events_train = np.array([0, 0, 1, 1, 2, 2, 3, 4])
Nc = np.sum(events_train == 0)
N = len(events_train)
p1 = Nc/N
p2 = 1 - p1
w = np.zeros(N)
w[:] = p2
ind_corres = np.where(events_train == 0)
w[ind_corres] = p1
w = torch.Tensor(0.5 / w)
print(w * predictions)

#print(zeros)

#print(zeros)
#indx_features = torch.nonzero(zeros == 1, as_tuple=True)
#print(indx_features)
#print(x_debug[indx_features])

tensor([[0.5023, 0.1447, 0.8197, 0.4740, 0.9310],
        [0.2122, 0.8518, 0.0427, 0.1589, 0.7650],
        [0.2654, 0.1662, 0.3603, 0.1758, 0.7407],
        [0.8739, 0.1780, 0.9209, 0.7400, 0.1299],
        [0.7337, 0.9146, 0.3225, 0.7295, 0.3336],
        [0.3338, 0.9430, 0.6573, 0.6252, 0.4321],
        [0.9042, 0.6314, 0.8092, 0.1849, 0.3303],
        [0.6423, 0.1659, 0.6461, 0.7879, 0.9842]])
tensor([0., 0., 1., 1., 2., 2., 3., 4.])
tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype=torch.int32) tensor([0, 0, 1, 1, 2, 2, 3, 4], dtype=torch.int32)
(tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype=torch.int32), tensor([0, 0, 1, 1, 2, 2, 3, 4], dtype=torch.int32))
tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1.0046, 0.4243, 0.1108, 0.1186, 0.2150, 0.4382, 0.1233, 0.6561])


In [3]:
import torch.nn as nn
A = torch.rand(2,3)
m = nn.Softmax(dim=1)
A = m(A)
print(A)

tensor([[0.3430, 0.3203, 0.3367],
        [0.2359, 0.4366, 0.3275]])
